In [146]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
%load_ext autoreload

%autoreload 2




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [147]:
from dc_egm import chop, polyline, aux_function, interpolate, upper_envelope
from ret import util, mutil, imutil, income, budget, mbudget, value_function, chpr, logsum, egm_step

# Model parameters (default)

In [148]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 10

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [149]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]

In [150]:
savingsgrid = np.linspace(0, mmax, ngridm)

In [154]:
# We are currently facing a problem since our result arrays do not have the same length. 
# Instead of a multidimensional array we should use a list tha

In [151]:
# Initialize containers

# Container for endogenous gridpoints of (beginning-of-period) assets
# and corresponding consumption
policy = np.full((ngridm + 1, 2, 2, Tbar), np.nan)

# Value functions
value = np.full((ngridm + 1, 2, 2, Tbar), np.nan)
# Handling of last period and first elements
# policy
policy[1:, 0, 0, Tbar-1] = savingsgrid
policy[1:, 0, 1, Tbar-1] = savingsgrid
policy[1:, 1, :, Tbar-1] = policy[1:, 0, :, Tbar-1]
policy[0, :, :, :] = 0.00
# value
value[2:, 0, :, Tbar-1] = util(policy[2:, 0, :, Tbar-1], 0, theta, duw)
value[2:, 1, :, Tbar-1] = util(policy[2:, 0, :, Tbar-1], 1, theta, duw)
value[0:2, :, :, Tbar -1] = 0.00
value[0, 0, :, :] = 0.00

In [152]:
for period in [23, 22, 21]:
    value = egm_step(value, policy, savingsgrid, quadstnorm, period, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

In [153]:
value[:, 1, 1, 23]

array([0.91252708, 1.42959855, 1.4328254 , 1.43597492, 1.4390499 ,
       1.44205296, 1.44498662, 1.44785329, 1.45065524, 1.45339466,
       1.45607365, 1.45869419, 1.46125819, 1.46376747, 1.46622379,
       1.46862882, 1.47098416, 1.47329135, 1.47555186, 1.4777671 ,
       1.47993843, 1.48206717, 1.48415454, 1.48620178, 1.48821001,
       1.49018037, 1.49211392, 1.49401169, 1.49587468, 1.49770383,
       1.49950008, 1.5012643 , 1.50299736, 1.50470007, 1.50637324,
       1.50801764, 1.509634  , 1.51122304, 1.51278546, 1.51432192,
       1.51583308, 1.51731955, 1.51878194, 1.52022084, 1.52163681,
       1.52303041, 1.52440215, 1.52575256, 1.52708214, 1.52839136,
       1.5296807 , 1.5309506 , 1.53220152, 1.53343386, 1.53464805,
       1.5358445 , 1.53702358, 1.53818567, 1.53933116, 1.54046038,
       1.54157369, 1.54267142, 1.54375391, 1.54482147, 1.5458744 ,
       1.54691302, 1.54793761, 1.54894845, 1.54994583, 1.55093002,
       1.55190127, 1.55285984, 1.55380598, 1.55473994, 1.55566